# Autonomous Trading Brain - Kaggle Training Pipeline

This notebook sets up the environment, installs dependencies (including TA-Lib and custom Deriv API), prepares the data (full scratch download), and runs the training loop on GPU.

## 1. Environment Setup & TA-Lib Installation
Compiling TA-Lib C-library (required for technical indicators).

In [ ]:
import os
import sys

# Install system build tools
!apt-get update > /dev/null
!apt-get install -y build-essential > /dev/null

# Download and compile TA-Lib
if not os.path.exists('/usr/lib/libta_lib.so'):
    print("Downloading TA-Lib source...")
    !wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
    !tar -xzf ta-lib-0.4.0-src.tar.gz
    
    print("Compiling TA-Lib (this takes ~2 mins)...")
    os.chdir('ta-lib')
    !./configure --prefix=/usr > /dev/null
    !make > /dev/null
    !make install > /dev/null
    os.chdir('..')
    !rm -rf ta-lib ta-lib-0.4.0-src.tar.gz
    print("TA-Lib installed successfully!")
else:
    print("TA-Lib already installed.")

## 2. Project Setup
Copying code from Input to Working directory to allow execution and modification.

In [ ]:
import shutil
from pathlib import Path

# CONFIG: Change this to match your uploaded dataset name
INPUT_DIR = Path('/kaggle/input/xtitan-project/xtitan') 
WORKING_DIR = Path('/kaggle/working/xtitan')

if not WORKING_DIR.exists():
    print(f"Copying project from {INPUT_DIR} to {WORKING_DIR}...")
    # Copy ignoring git/venv/cache if they exist
    shutil.copytree(INPUT_DIR, WORKING_DIR, 
                    ignore=shutil.ignore_patterns('venv', '__pycache__', '.git', '*.optimize', 'data_cache'))
    print("Project setup complete.")
else:
    print("Project already in working directory.")

os.chdir(WORKING_DIR)

## 3. Install Python Dependencies
Installing requirements and custom API.

In [ ]:
!pip install TA-Lib
!pip install -r requirements.txt

# CORE: Install local custom Deriv API
print("Installing custom python-deriv-api...")
!pip install ./python-deriv-api

## 4. Configuration
Edit the cell below to configure your training parameters. 
**Run this cell to save the `.env` file.**

In [ ]:
%%writefile .env
TRADING__SYMBOL=R_100
TRADING__STAKE_AMOUNT=10.0

# Probability Thresholds
THRESHOLDS__CONFIDENCE_THRESHOLD_HIGH=0.65
THRESHOLDS__LEARNING_THRESHOLD_MIN=0.45
THRESHOLDS__LEARNING_THRESHOLD_MAX=0.60

# Model Hyperparameters
HYPERPARAMS__LEARNING_RATE=0.0005
HYPERPARAMS__BATCH_SIZE=64
HYPERPARAMS__LSTM_HIDDEN_SIZE=128
HYPERPARAMS__CNN_FILTERS=64
HYPERPARAMS__LATENT_DIM=32

# Data Shapes
DATA_SHAPES__SEQUENCE_LENGTH_TICKS=60
DATA_SHAPES__SEQUENCE_LENGTH_CANDLES=60

# Optional: Set API Token here directly, OR use Kaggle Secrets (see next cell)
# DERIV_API_TOKEN=your_token_here

### 4.1 Load Secrets (Optional)
If you have set `DERIV_API_TOKEN` in Kaggle Secrets (Add-ons -> Secrets), this block will append it to the configuration securely.

In [ ]:
from kaggle_secrets import UserSecretsClient

try:
    user_secrets = UserSecretsClient()
    api_token = user_secrets.get_secret("DERIV_API_TOKEN")
    
    with open(".env", "a") as f:
        f.write(f"\nDERIV_API_TOKEN={api_token}\n")
    print("✅ Loaded DERIV_API_TOKEN from Kaggle Secrets and appended to .env")
except Exception:
    print("ℹ️ No Kaggle Secret 'DERIV_API_TOKEN' found. (Ignore this if you set it manually above)")

## 5. Data Preparation (Fresh Download)
Downloading 12 months of historical data from scratch. Previous data is cleared to ensure integrity.

In [ ]:
import shutil
from pathlib import Path

DATA_DIR = Path('data_cache')

# CLEAN START: Remove existing data cache if any
if DATA_DIR.exists():
    print("Cleaning existing data cache...")
    shutil.rmtree(DATA_DIR)

print("Downloading 12 months of data (This may take a while)...")
# Force download with --no-resume
!python scripts/download_data.py --months 12 --symbol R_100 --output data_cache --no-resume

## 6. System Validation
Running data integrity checks and pre-flight validation.

In [ ]:
print("--- Final Integrity Check ---")
!python scripts/final_integrity_check.py

print("\n--- Pre-Training Validation ---")
!python pre_training_validation.py

## 7. Training
Running the main training loop.

In [ ]:
# GPU Validation
!nvidia-smi

# Clear previous logs if needed
# !rm -rf logs/

# Run training
!python scripts/train.py

## 8. Artifact Archival
Zip results for download.

In [ ]:
import shutil
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
archive_name = f"training_artifacts_{timestamp}"

print("Archiving checkpoints and logs...")
shutil.make_archive(archive_name, 'zip', root_dir='.', base_dir='checkpoints')
shutil.make_archive(f"{archive_name}_logs", 'zip', root_dir='.', base_dir='logs')

print(f"Created {archive_name}.zip and {archive_name}_logs.zip")